# Programa 1

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

## Métodos de Runge-Kutta-Merson y de grado 4

### Rubrica
|Elemento |Pts.|
|---|---|
|Cambia PVI |3|
|Lee valores iniciales |2|
|Pide $h$ y tolerancia |2|
|Presenta tabla $(x,y)$ correcta [R-K-4] |4|
|Presenta tabla $(x,y)$ correcta [R-K-M] |4|
|Da opción de presentar tabla (K's) |3|
|Compara métodos con diferencia entre ellos |2|
|Permite volver a ejecutar y cambiar valores iniciales |2|
|Aplica control de error (Extra)|2|


## Código

In [3]:
class RungeKuttas(object):
    
    # Encabezados
    header=list(("x","y","k1","k2","k3","k4","k5","Error"))

    # Constructor
    def __init__(self, initialValue:list, point:float, h_value:float, error:float = 0.0, equation:str="0") -> None:
        self._initialValue = initialValue
        self.h_value = h_value
        self.error = error
        self.solTab = np.zeros((1,8),dtype=float)
        self.point = point
        self.equation = equation

    # Lo que muestra al llamar la función print
    def __str__(self:any) -> str:
        self.rkmMethod()
        return "Objeto de tipo Método Runge-Kutta"
    
    #region Método de Runge-Kutta-4
    def rk4(self:any):
        self.firstRowOrder4()
        self.iterationsOrder4()
        return pd.DataFrame(self.solTab, columns=self.header)[['x','y','k1','k2','k3','k4','Error']]
    
    # Método para crear la primera iteración de RK4
    def firstRowOrder4(self:any) -> None:
        x,y = self._initialValue[0], self._initialValue[1]
        self.solTab[0,0] = x
        self.solTab[0,1] = y
        self.solTab[0,2] = self.k1(x,y)
        self.solTab[0,3] = self.k2(x,y,self.solTab[0,2])
        self.solTab[0,4] = self.k3(x,y,self.solTab[0,3])
        self.solTab[0,5] = self.k4(x,y,self.solTab[0,2],self.solTab[0,3],self.solTab[0,4])

    # Método para crear el resto de las iteraciones de RK4
    def iterationsOrder4(self:any) -> None:
        rango = int(np.abs((self._initialValue[0]-self.point)/self.h_value))
        for i in range(0,rango):
            yn = np.zeros(shape=(1,8))
            x,y = self.solTab[i,0] + self.h_value,self.solTab[i,1]
            yn[0,0] = x
            yn[0,1] = self.get_y(y=y, k1=self.solTab[i,2], k2=self.solTab[i,3], k3=self.solTab[i,4], k4=self.solTab[i,5])
            y = yn[0,1]
            yn[0,2] = self.k1(x,y)
            yn[0,3] = self.k2(x,y,yn[0,2])
            yn[0,4] = self.k3(x,y,yn[0,3])
            yn[0,5] = self.k4(x,y,yn[0,2],yn[0,3],yn[0,4])
            self.solTab = np.append(self.solTab,yn, axis=0)
            del yn,x,y

    # Métodos para obtener las k del metodo de RK4
    def k1(self, x:float, y:float) -> float:
        return self.fdexy(x,y)

    def k2(self, x:float, y:float, k1:float) -> float:
        return self.fdexy(x + self.h_value/2, y + k1 * self.h_value /2)

    def k3(self, x:float, y:float, k2:float) -> float:
        return self.fdexy(x + self.h_value/2, y + k2 * self.h_value /2)

    def k4(self, x:float, y:float, k1:float, k2:float, k3:float) -> float:
        return self.fdexy(x + self.h_value, y + k3 * self.h_value )

    def get_y(self,y:float, k1:float, k2:float, k3:float, k4:float) -> float:
        """Return the next y with a global error of O(h^3)"""
        return y + (k1 + 2 * k2 + 2 * k3 + k4) * self.h_value/6
    
    #endregion

    #region Método de Runge-Kutta-Merson
    def rkmMethod(self:any) -> pd.DataFrame:
        self.firstRowMerson()
        self.iterationsMerson()
        return pd.DataFrame(self.solTab, columns=self.header)[['x','y','k1','k2','k3','k4','k5','Error']]

    # Método para crear la primera iteración de Merson
    def firstRowMerson(self:any) -> None:
        x,y = self._initialValue[0], self._initialValue[1]
        self.solTab[0,0] = x
        self.solTab[0,1] = y
        self.solTab[0,2] = self.mersonk1(x,y)
        self.solTab[0,3] = self.mersonk2(x,y,self.solTab[0,2])
        self.solTab[0,4] = self.mersonk3(x,y,self.solTab[0,2],self.solTab[0,3])
        self.solTab[0,5] = self.mersonk4(x,y,self.solTab[0,2],self.solTab[0,3],self.solTab[0,4])
        self.solTab[0,6] = self.mersonk5(x,y,self.solTab[0,2],self.solTab[0,3],self.solTab[0,4],self.solTab[0,5])

    # Método para crear el resto de las iteraciones de Merson
    def iterationsMerson(self) -> None:
        rango = int(np.abs((self._initialValue[0]-self.point)/self.h_value))
        for i in range(0,rango):
            yn = np.zeros(shape=(1,8))
            x,y = self.solTab[i,0] + self.h_value,self.solTab[i,1]
            yn[0,0] = x
            yn[0,1] = self.get_yMerson(y=y, k1=self.solTab[i,2], k3=self.solTab[i,4], k4=self.solTab[i,5], k5=self.solTab[i,6])
            y = yn[0,1]
            yn[0,2] = self.mersonk1(x,y)
            yn[0,3] = self.mersonk2(x,y,yn[0,2])
            yn[0,4] = self.mersonk3(x,y,yn[0,2],yn[0,3])
            yn[0,5] = self.mersonk4(x,y,yn[0,2],yn[0,3],yn[0,4])
            yn[0,6] = self.mersonk5(x,y,yn[0,2],yn[0,3],yn[0,4],yn[0,5])
            self.solTab = np.append(self.solTab,yn, axis=0)
            del yn,x,y

    # Esta función recibirá una cadena que representará la función, usaremos la función "eval"
    def fdexy(self,x:float, y:float) -> float:
        """This is the equation we want to get the numeric solution"""
        # Notita: las variables x, y si se ocupan ¡aunque no lo parezca!
        return eval(self.equation)

    # Métodos para obtener las k del metodo de Merson
    def mersonk1(self, x:float, y:float) -> float:
        return self.h_value * self.fdexy(x,y)

    def mersonk2(self, x:float, y:float, k1:float) -> float:
        return self.h_value * self.fdexy(x + self.h_value/3, y + k1/3)

    def mersonk3(self, x:float, y:float, k1:float, k2:float) -> float:
        return self.h_value * self.fdexy(x + self.h_value / 3, y + k1/6 + k2/6)

    def mersonk4(self, x:float, y:float, k1:float, k2:float, k3:float) -> float:
        return self.h_value * self.fdexy(x + self.h_value / 2, y + k1 / 8 + 3/8 * k3)

    def mersonk5(self,x:float,y:float, k1:float, k2:float, k3:float, k4:float) -> float:
        return self.h_value * self.fdexy(x + self.h_value, y + k1/2 - 3/2 * k3 + 2*k4)

    def mersonk6(self, x:float, y:float, k1:float, k2:float, k3:float, k4:float, k5:float) -> float:
        return self.h_value * self.fdexy(x + self.h_value/2, y - 8/27*k1 + 2*k2 - 3544/2565*k3 + 1859/4104*k4 - 11/40*k5)

    # Regresa la y de Merson
    def get_yMerson(self,y:float, k1:float, k3:float, k4:float, k5:float) -> float:
        """Return the next y with a global error of O(h^4)"""
        return y + (k1 + 4*k4 + k5)/6
    #endregion


    def controlE(hmax:float,hmin:float,tol:float):
        haux=None
        print('Ingresa h maxima: ')
        hmax = input()
        print('Ingresa h minima: ')
        hmin= input()
        print('Ingresa la tolerancia: ')
        tol = input()
        E = ((k1/360)-((128/4275)*k2)-((2197/75240)*k4)+(k5/50)+((2/55)*k6))/hmax
        Q = 0.84*((tol/E)**(1/4))
        if E<= tol :
            return RungeKuttas()
        if Q<=0.1:
           return haux == 0.(hmax)
        if Q >= 4:
            return haux==4*(hmax)
        else:
            return(haux==Q*hmax)
        if haux>hmax :
            haux=hmax
        else:
            exit

### Tabla con el método de grado 4

In [4]:
tab1=RungeKuttas(initialValue=(0,3),h_value=0.5, point=7, equation="-0.06*y**(1/2)").rk4()
tab1

,x,y,k1,k2,k3,k4,Error
0,0.0,3.000000,-0.103923,-0.103472,-0.103474,-0.103023,0.0
1,0.5,2.948263,-0.103023,-0.102572,-0.102574,-0.102123,0.0
2,1.0,2.896977,-0.102123,-0.101672,-0.101674,-0.101223,0.0
3,1.5,2.846140,-0.101223,-0.100772,-0.100774,-0.100323,0.0
4,2.0,2.795754,-0.100323,-0.099872,-0.099874,-0.099423,0.0
5,2.5,2.745817,-0.099423,-0.098972,-0.098974,-0.098523,0.0
6,3.0,2.696331,-0.098523,-0.098072,-0.098074,-0.097623,0.0
7,3.5,2.647294,-0.097623,-0.097172,-0.097174,-0.096723,0.0
8,4.0,2.598708,-0.096723,-0.096272,-0.096274,-0.095823,0.0
9,4.5,2.550571,-0.095823,-0.095372,-0.095374,-0.094923,0.0


### Tabla con el método de R-K-M

In [5]:
tab2=RungeKuttas(initialValue=(0,3),h_value=0.5, point=7, equation="-0.06*y**(1/2)").rkmMethod()
tab2

,x,y,k1,k2,k3,k4,k5,Error
0,0.0,3.000000,-0.051962,-0.051811,-0.051812,-0.051737,-0.051512,0.0
1,0.5,2.948263,-0.051512,-0.051361,-0.051362,-0.051287,-0.051062,0.0
2,1.0,2.896977,-0.051062,-0.050911,-0.050912,-0.050837,-0.050612,0.0
3,1.5,2.846140,-0.050612,-0.050461,-0.050462,-0.050387,-0.050162,0.0
4,2.0,2.795754,-0.050162,-0.050011,-0.050012,-0.049937,-0.049712,0.0
5,2.5,2.745817,-0.049712,-0.049561,-0.049562,-0.049487,-0.049262,0.0
6,3.0,2.696331,-0.049262,-0.049111,-0.049112,-0.049037,-0.048812,0.0
7,3.5,2.647294,-0.048812,-0.048661,-0.048662,-0.048587,-0.048362,0.0
8,4.0,2.598708,-0.048362,-0.048211,-0.048212,-0.048137,-0.047912,0.0
9,4.5,2.550571,-0.047912,-0.047761,-0.047762,-0.047687,-0.047462,0.0
